# Demonstration of USB communication with the Versalase laser
This notebook demonstrates communication with the Stradus Versalase laser over a USB link. The underlying text protocol is documented (described for RS232 comms) in section 9 of the manufacturer's Stradus Versalase™ User Manual supplied with the laser. However there is no documentation on how to communicate with it over USB (except using their own Windows GUI program). I reverse-engineered the protocol (details at https://jmtayloruk.github.io/tools/2024/07/06/reverse-engineering-laser-control/), and this notebook implements the protocol in Python code.

## To install pyusb support to run the Python code in this notebook
`brew install libusb`

`pip install pyusb`

For M1 suport for libusb, run the following (see https://github.com/pyusb/pyusb/issues/355)
`ln -s /opt/homebrew/lib/libusb-1.0.0.dylib //usr/local/lib/libusb.dylib`

## Other optional notes

### Implementing this in C/Cocoa code on OS X
If I do want to eventually implement this in C/Cocoa code, the relevant information is here 
https://developer.apple.com/library/archive/documentation/DeviceDrivers/Conceptual/USBBook/USBDeviceInterfaces/USBDevInterfaces.html#//apple_ref/doc/uid/TP40002645-TPXREF101
It looks like this code is actually quite close to what I need, so it shouldn’t be too much of a faff to code this. 

### To run Wireshark on OS X (not necessary, but included here as a reminder to myself)
To access the USB interface you need to run
`sudo ifconfig XHC20 up`

For that to work on 10.13 or up, you need to disable system integrity protection: cmd-R to restart in recovery mode, launch terminal, type `csrutil disable` (from https://developer.apple.com/library/archive/documentation/Security/Conceptual/System_Integrity_Protection_Guide/ConfiguringSystemIntegrityProtection/ConfiguringSystemIntegrityProtection.html#//apple_ref/doc/uid/TP40016462-CH5-SW1)

### What is the protocol?

I don’t think it’s a FTDI usb/serial adaptor - that seems to send different packets over the USB link, including (from my recollection…) ones over the bulk in/out endpoints.
Neither is it USBTMC (Richard Bowman’s suggestion). That too does data transfers over bulk in/out endpoints (which we don't see here). Indeed, FTDI could well be using that exact protocol (I didn't look). It's not HID as I do not see "boot" messages being sent.

It does not show up as a COM port (or indeed any kind of device, as far as I can see) in the Windows Device Manager.

In [1]:
import usb.core
import sys, time

In [2]:
def GetA1Response(timeout=2.0, minLength=2,timeOutAcceptable=False):
    t0 = time.time()
    didFind = None
    firstSeen = None
    while time.time() < t0+timeout:
        result = dev.ctrl_transfer(0xc0, 0xa1, 0x0000, 0, 256)
        sret = ''.join([chr(x) for x in result])
        if len(sret) > minLength:
            return sret
        elif len(sret) > 0:
            if firstSeen is None:
                firstSeen = time.time() - t0
            didFind = sret
        time.sleep(0.01)
    if timeOutAcceptable:
        return ''
    print(" [Read timed out]")
    if didFind is not None:
        print(f"  (but did see {bytes(sret,'utf8')} after {firstSeen:.3f})")
    raise TimeoutError

def SendA0TextCommand(cmd, logLevel=0):
    if logLevel >= 2:
        print(f" Send command {cmd}")
    # This sends the command to the laser
    dev.ctrl_transfer(0x40, 0xa0, 0x0000, 0, cmd+'\r')

def GetA2(logLevel=0):
    result = dev.ctrl_transfer(0xc0, 0xa2, 0x0000, 0, 1)
    if logLevel >= 2:
        if (len(result) == 1):
            print(f" A2 read got: {result[0]}")
        else:
            print(f" A2 read got unexpected: {bytes(result, 'utf8')}")
    return result[0]

def GetA1(printType):
    resp = ReadResponse(minLength=0)
    print(f" Response: {bytes(resp)}") # Expecting '\r\n'

def SendA3(logLevel):
    if logLevel >= 2:
        print(" Send A3")
    dev.ctrl_transfer(0x40, 0xa3, 0x0000, 0, 0)
    
# Main utility function to send a text command to the laser, and receive the response
def SendCommand(cmd, logLevel=0):
    result = None
    SendA0TextCommand(cmd+'\r',logLevel)

    # It seems that this initial A1 query has to be done, before we query A2.
    # However, this A1 response may be empty,
    # and we need to tolerate that without raising a timeout exception
    resp = GetA1Response(minLength=0, timeout=0.5, timeOutAcceptable=True)      
    if logLevel >= 2:
        if resp == '':
            print(f" Initial A1 read timed out")
        else:
            if (len(resp) >= 2) and (resp[0:2] == '\r\n'):
                resp = resp[2:]
            print(f" Initial A1 read: '{resp}'")
        
    # There might be a delay (sometimes more than 0.5s if changing a laser parameter)
    # before the response is available.
    # We should always expect some sort of a response (I think) even if it's a blank.
    # So, we initially poll until we get a response of 1 to an A2 query
    t0 = time.time()
    initiallyZero = False
    while GetA2(0) == 0:
        initiallyZero = True
        if time.time() > t0 + 5:
            if logLevel >= 1:
                print(" A2 reads never returned 1")
                break
    if initiallyZero and (logLevel >= 2) and (GetA2(0) == 1):
        print(f" A2 read took {time.time()-t0:.3f}s to return 1")
    
    while GetA2(logLevel) == 1:
        # Message available to read
        # Read it
        resp = GetA1Response(minLength=0)[2:]   # Should decide what to do if response is anomalous (too short)
        if logLevel >= 2:
            print(f" A1 read got answer '{resp}'")
        if resp == '':
            if logLevel >= 2:
                print(f" A1 read got blank prompt")
        elif resp == 'Stradus> ':
            if logLevel >= 2:
                print(f" A1 read got stradus prompt")
        else:
            result = resp
        # Acknowledge that we have read this message
        SendA3(logLevel)
    if logLevel >= 1:
        print(f"Sent {cmd}, got response '{result}'")
    return result

# Utility functions to send a query and parse the response as a number
def ParseQuery(cmd):
    response = SendCommand(cmd)
    return response[len(cmd)+1:]
def ParseFloatQuery(cmd):
    return float(ParseQuery(cmd))
def ParseIntQuery(cmd):
    return int(ParseQuery(cmd))

# Example code control of the laser

Note that if the interlock is open then the LE=1 command sometimes fails to provide a response.
The documentation states it is not allowed when interlock is open - but implies we should get an LE=0 response.
Sometimes we see no response at all, and that seems to mess up comms for a period of a few seconds.
Note that because we have a 5 second timeout waiting for the initial A2 read,
that allows enough time for the comms to reset. However if we use a shorted timeout (e.g. 2s) then
subsequent commands will also be messed up

In [3]:
# Attach to the device according to its vendor and product ID
dev = usb.core.find(idVendor=0x201a, idProduct=0x0003)
if dev is None:
    raise ValueError('Our device is not connected')

In [6]:
# Demonstration of usage: query information about the installed lasers
for las in ['a','b','c','d']:
    laserInfo = SendCommand(f'{las}.?li')
    print(f"Laser {las} information (undocumented command): {laserInfo}")
    if laserInfo is not None:
        # This laser is actually installed in the laser box - learn more about it
        print(f"Laser wavelength: {ParseFloatQuery(f'{las}.?lw'):.1f}")
        print(f"Maximum output power: {ParseFloatQuery(f'{las}.?maxp'):.2f}")
        print(f"Maximum rated power: {ParseFloatQuery(f'{las}.?rp'):.2f}")
        print(f"Laser emitting: {ParseIntQuery(f'{las}.?le')}")
        print(f"Laser power setting: {ParseFloatQuery(f'{las}.?lps'):.2f}")
        print(f"Measured power output: {ParseFloatQuery(f'{las}.?lp'):.2f}")
    print("")


Laser a information (undocumented command): None

Laser b information (undocumented command): B.?LI=VL03144D11, 11078, 561nm, 50mW, C
Laser wavelength: 561.0
Maximum output power: 50.00
Maximum rated power: 50.00
Laser emitting: 1
Laser power setting: 50.00
Measured power output: 0.00

Laser c information (undocumented command): C.?LI=VL03144D03, 11074, 488nm, 50mW, C
Laser wavelength: 490.0
Maximum output power: 51.00
Maximum rated power: 50.00
Laser emitting: 1
Laser power setting: 50.00
Measured power output: 0.10

Laser d information (undocumented command): D.?LI=VL03144D07, 11070, 405nm, 100mW, C
Laser wavelength: 402.0
Maximum output power: 101.00
Maximum rated power: 100.00
Laser emitting: 1
Laser power setting: 50.00
Measured power output: 0.10



In [7]:
# Apply the default settings that I want for each laser
# Note that LP is documented to respond with *measured* LP. 
# If it fails, it actually seems to respond with *set* LPS, not measured LP
response = SendCommand(f'a.epc=1',logLevel=1)
response = SendCommand(f'a.le=0',logLevel=1)
response = SendCommand(f'a.lp=1',logLevel=1)
response = SendCommand(f'b.le=1',logLevel=1)
response = SendCommand(f'b.epc=0',logLevel=1)
response = SendCommand(f'b.lp=50',logLevel=1)
response = SendCommand(f'c.le=1',logLevel=1)
response = SendCommand(f'c.epc=0',logLevel=1)
response = SendCommand(f'c.lp=50',logLevel=1)
response = SendCommand(f'd.le=1',logLevel=1)
response = SendCommand(f'd.epc=0',logLevel=1)
response = SendCommand(f'd.lp=50',logLevel=1)

Sent a.epc=1, got response 'None'
Sent a.le=0, got response 'None'
Sent a.lp=1, got response 'None'
Sent b.le=1, got response 'B.LE=1'
Sent b.epc=0, got response 'B.EPC=0'
Sent b.lp=50, got response 'B.LP=0.0'
Sent c.le=1, got response 'C.LE=1'
Sent c.epc=0, got response 'C.EPC=0'
Sent c.lp=50, got response 'C.LP=0.1'
Sent d.le=1, got response 'D.LE=1'
Sent d.epc=0, got response 'D.EPC=0'
Sent d.lp=50, got response 'D.LP=0.1'
